In [16]:
features = {"RS":"rolled-in scale", "Pa":"patches", "Cr":"crazing", "PS":"pitted surface", "In":"inclusion", "Sc":"scratches"}

In [54]:
!pip install tensorflow
!pip install keras

  Using cached https://files.pythonhosted.org/packages/de/f0/96fb2e0412ae9692dbf400e5b04432885f677ad6241c088ccc5fe7724d69/tensorflow-1.14.0-cp36-cp36m-manylinux1_x86_64.whl
Exception:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/pip/basecommand.py", line 215, in main
    status = self.run(options, args)
  File "/opt/conda/lib/python3.6/site-packages/pip/commands/install.py", line 335, in run
    wb.build(autobuilding=True)
  File "/opt/conda/lib/python3.6/site-packages/pip/wheel.py", line 756, in build
    self.requirement_set.prepare_files(self.finder)
  File "/opt/conda/lib/python3.6/site-packages/pip/req/req_set.py", line 380, in prepare_files
    ignore_dependencies=self.ignore_dependencies))
  File "/opt/conda/lib/python3.6/site-packages/pip/req/req_set.py", line 620, in _prepare_file
    session=self.session, hashes=hashes)
  File "/opt/conda/lib/python3.6/site-packages/pip/download.py", line 821, in unpack_url
    hashes=hashes
  File "/opt/c

In [38]:
import os
import numpy as np

In [39]:
FOLDER_IMAGE = "data"

In [55]:
def load_images(): 
    y = []
    x = []
    for filename in os.listdir(FOLDER_IMAGE):
        image_path = os.path.join(FOLDER_IMAGE, filename)
        image = imread(image_path)
        x.append(image)
        y.append(filename.split("_")[0])
    y = np.vstack(y)
    x = np.stack(x)
    return x, y
x, y = load_images()

In [56]:
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate

def get_siamese_model(input_shape):
    """
        Model architecture
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (7,7), activation='relu',
                     kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(MaxPooling2D())
    model.add(Conv2D(256, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net
model = get_siamese_model((105, 105, 1))


Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'